In [6]:
#Imports
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openweathermapy.core as ow
from citipy import citipy 
import random
import requests as req

#OpenWeatherMap API Key
api_key = "26fea83d1259ec5c651394906a953b54"
base_url = "http://api.openweathermap.org/data/2.5/weather"


In [7]:
#Random selection of 500 unique cities based on lat/long coordinates
#Truncate random coordinates to 6 decimal points each
def truncate(x, d): #Source for truncate() function: https://stackoverflow.com/a/20545024
    return int(x*(10.0**d))/(10.0**d)
    #x = input
    #d = decimal places to truncate to

cityList = []
selected_cites = []
uniqueCityCount = 0

while (uniqueCityCount < 10): 
    random_lat = truncate(random.uniform(-90,90),6)
    random_long = truncate(random.uniform(-180,180),6)
    cityToCheck = citipy.nearest_city(random_lat, random_long).city_name
    #Pass query to OpenWeatherMap API to check if city is in database
    
    query_city = cityToCheck
    query_url = base_url + "?apikey=" + api_key + "&q=" + query_city
    response = req.get(query_url)
    json_data = response.json()
    
    #Prevent duplicates from being appended to the cityList (before dataframe creation)
    #While also not adding the specified city if not in OWM database
    if cityToCheck not in cityList and json_data['cod'] != '404':
        cityList.append(cityToCheck)
        uniqueCityCount = uniqueCityCount+ 1 
        
#Create dataframe with uniqueCity names        
raw_cities_df = pd.DataFrame({"City Name": cityList})

#Not needed if duplicates removed before dataframe creation: 
#raw_cities_df = raw_cities_df.drop_duplicates() 

#Creation of column for city number
raw_cities_df['City Number'] = raw_cities_df.index + 1
raw_cities_df= raw_cities_df[['City Number','City Name']]
cn = pd.to_numeric(raw_cities_df['City Number'], errors='raise', downcast=None)

In [8]:

raw_cities_df.head(25)
#print(raw_cities_df['City Number'])

,City Number,City Name
0,1,lasa
1,2,hamilton
2,3,te anau
3,4,mataura
4,5,mahebourg
5,6,hithadhoo
6,7,khatanga
7,8,juneau
8,9,pacific grove
9,10,rikitea


In [14]:
settings = {"units": "imperial","appid": api_key}
from time import sleep

#Print parameters being requested from owm API before retrival
print("Base URL for API requests:... " + ow.BASE_URL)
print("Parameters being requested per city:... ")
summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
print(summary)

#Get current weather for each city in cityList
cityCount = 1
for city in raw_cities_df["City Name"]:
    sleep(1.2)
    try:
        print("Processing city " + str(cityCount)+ " of " +str(len(raw_cities_df["City Name"])) + ", "+ city +":") 
        current_city= ow.get_current(city, **settings)
        sleep(1.2)
        #Leveraging OpenWatherMapy to parse the response with current_city(*summary)
        #summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
        summary_output = current_city(*summary)
        print(str(summary_output))
    
    except Exception as ex:
        print(ex)
        cityCount = cityCount - 1
    cityCount = cityCount + 1

    #openweathermap complete city list:
    #http://openweathermap.org/help/city_list.txt 
    #one method ---> check against list
    #another method ---> skip if error returned 
    
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
    
    

Base URL for API requests:... http://api.openweathermap.org/data/2.5/
Parameters being requested per city:... 
['name', 'coord.lat', 'coord.lon', 'main.temp', 'main.humidity', 'wind.speed', 'clouds.all']
Processing city 1 of 10, lasa:
('Lasa', 34.92, 32.53, 51.21, 93, 6.93, 75)
Processing city 2 of 10, hamilton:
('Hamilton', 32.3, -64.78, 68, 88, 18.34, 90)
Processing city 3 of 10, te anau:
('Te Anau', -45.41, 167.72, 61.64, 100, 3.49, 92)
Processing city 4 of 10, mataura:
('Mataura', -46.19, 168.86, 65.42, 98, 13.67, 92)
Processing city 5 of 10, mahebourg:
('Mahebourg', -20.41, 57.7, 75.81, 100, 22.84, 88)
Processing city 6 of 10, hithadhoo:
('Hithadhoo', -0.6, 73.08, 81.17, 100, 3.83, 64)
Processing city 7 of 10, khatanga:
('Khatanga', 71.98, 102.47, -47.36, 0, 11.99, 0)
Processing city 8 of 10, juneau:
('Juneau', 58.3, -134.42, 43.79, 87, 16.11, 40)
Processing city 9 of 10, pacific grove:
('Pacific Grove', 36.62, -121.92, 63.32, 82, 4.7, 1)
Processing city 10 of 10, rikitea:
('Rikit

In [10]:
#Put requested data into a dictionary that contains a list (intention to append to dataframe)
views = {"summary":["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]}
print(current_city(*views["summary"]))

#raw_cities_df[["summary"]]
#pd.DataFrame_fromt_dict
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html
#Mapping ----> http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.Series.map.html
#https://stackoverflow.com/questions/26666919/python-pandas-add-column-in-dataframe-from-list
#https://stackoverflow.com/questions/42632470/how-to-add-dictionaries-to-a-dataframe-as-a-row
#Documentation on OWM wrapper ---> https://github.com/crazycapivara/openweathermapy

('Rikitea', -23.12, -134.97, 80.31, 99, 15.12, 0)
